In [2]:
import pandas as pd
import os
from os.path import join

In [24]:
def distribute_evenly(df, group_count:int, criteria_key="새채무자키", criteria_amounts="현재원금"):
    """df, 분배할 그룹수, 기준 컬럼명 입력(새채무자키,주민번호...), 기준금액"""
    # 채무자별로 그룹화
    groupby_newdebtkey_df = df.groupby(criteria_key).agg({criteria_amounts:'sum'})
    # OPB 정렬
    sortedby_amounts_df = groupby_newdebtkey_df.sort_values(by=criteria_amounts, ascending=False)

    # 중간데이터 저장용 리스트 생성(3개의 그룹가정하고 설명)
    # 인덱스 저장할 리스트, 그룹수만큼 [[],[],[]]
    groups = [[] for _ in range(group_count)] 
    # 그룹별 금액 합계용 : 0이 그룹수만큼 담긴 리스트 [0,0,0] 
    # 그룹함수를 적용해서 opb가 실수타입으로 바뀌었기 때문에 반드시 실수형으로 초기화 해줘야 함
    group_amounts = [0.0] * group_count 
    # 그룹의 건수 저장용 : 0이 그룹수만큼 담긴 리스트 [0,0,0]
    group_counts = [0] * group_count 

    for idx, row in sortedby_amounts_df.iterrows():
        # 가장 작은 금액을 가진 group_amounts의 인덱스 = 가장 작은 금액을 갖는 그룹명
        group_to_allot = group_amounts.index(min(group_amounts))
        groups[group_to_allot].append(idx) # 인덱스 저장 : 정렬하면 idx도 따라감
        group_amounts[group_to_allot] += row[criteria_amounts] # 배정한 그룹 OPB 집계
        group_counts[group_to_allot] += 1
        
    for i, group in enumerate(groups):
        groupby_newdebtkey_df.loc[group, '그룹'] = i

    return groupby_newdebtkey_df, group_amounts, group_counts

In [27]:
# 파일 읽기
wd = r"c://users/SL/Desktop/"
filename = "실효폐지 배정_230608"
df = pd.read_excel(join(dir, filename+".xlsx"))
# 키열 문자열로
columns_with_key = list(df.columns[df.columns.str.contains("키")])
df[columns_with_key] = df[columns_with_key].astype(str)

In [28]:
# 균등 분배 수행
grouped_df, group_amounts, group_counts = distribute_evenly(df, 3, criteria_amounts='OPB')

# 결과 확인
for i in range(len(group_amounts)):
    print(f"Group {i+1}: Amount = {group_amounts[i]}, Count = {group_counts[i]}")

# 결과 저장
grouped_df.to_excel(join(wd, filename + '_분배표.xlsx'))

Group 1: Amount = 1940165031.0, Count = 307
Group 2: Amount = 1940187901.0, Count = 308
Group 3: Amount = 1940156195.0, Count = 308
